In [25]:
import pandas as pd
from nltk import *

In [26]:
from framework import *
from ingredients import *
from helpers import *
from AutoCrawl import *
from steps import *

In [71]:
def getVerbs(text):
    methodlist = set()
    #sentences = text.split("\n")
    for sentences in text:
        method = sentence_verbs(sentences)
        methodlist = methodlist.union(set(method))
    return list(methodlist)

def sentence_verbs(sentence):
    vblist = []
    subsentencelist = re.split('[;.:]', sentence)
    for subs in subsentencelist:
        vb = subsentence_verbs(subs)
        vblist = vblist + vb
    return vblist
        
def subsentence_verbs(subsentence):
    tokens = word_tokenize(subsentence)
    tagtup = nltk.pos_tag(tokens)
    methodlist = []
    for i in range(len(tagtup)):
        if i == 0 and tagtup[0][1] not in ["RB", "DT"]:
            methodlist.append(tagtup[i][0].lower())
        elif tagtup[i][1] in ["VB", "VBZ"] and tagtup[i][0] not in ['is', 'are', 'am', 'a', 'an']:
            methodlist.append(tagtup[i][0].lower())
    return methodlist

def getLOfMethods(fname):
    with open(fname) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    content = [x.lower() for x in content]
    return content

def isCookingMethod(verblist, allmethods):
    methods = []
    for verb in verblist:
        if verb in allmethods:
            methods.append(verb)
    return methods

def isPrepMethod(verblist, cookingmethod):
    prepmethod = list(set(verblist) - set(cookingmethod))
    return prepmethod

def mainIngCookingMethod(mainingr, sentences):
    cookingList = set()
    for sentence in sentences:
        if mainingr in sentence.lower():
            method = sentence_verbs(sentence)
            cooking = isCookingMethod(method, allmethods)
            cookingList = cookingList.union(cooking)

    return list(cookingList)

def parse_max_cook_time(get_times_result):
    timevalue = 0
    if isinstance(get_times_result, str) == True:
        splitted = get_times_result.split(" ")
        if 'hour' in get_times_result:
            timevalue += 3600 * int(splitted[len(splitted)-2])
        elif 'minute' in get_times_result:
            timevalue += 60 * int(splitted[len(splitted)-2])
        elif 'second' in get_times_result:
            timevalue += int(splitted[len(splitted)-2])
        else:
            timevalue += 0
    return timevalue

def longestCookingMethod(sentences):
    minimumtime = 0
    for sentence in sentences:
        cookingtimestring = get_times(sentence)
        cookingtimestep = parse_max_cook_time(cookingtimestring)
        if cookingtimestep > 0 and cookingtimestep > minimumtime:
            maincookingverb = sentence_verbs(sentence)
            maincookingmethod = isCookingMethod(maincookingverb, allmethods)
            if len(maincookingmethod) > 0:
                minimumtime = cookingtimestep
    return maincookingmethod

mainingr = 'cauliflower'
def ingredientMentionCount(mainingr, sentences):
    mentioned = 0
    countDict = dict()
    for sentence in sentences:
        if mainingr in sentence.lower():
            mentioned += 1
    countDict[mainingr] = mentioned 
    return countDict

ingredientMentionCount(mainingr, sentences)

In [78]:
list_methods = getLOfMethods('ListOfMethods.txt')
allmethods = [item.lower() for item in list_methods]
recipeText = GetData("https://www.allrecipes.com/recipe/219981/crab-stuffed-lobster-tail/")
recipeDirecton = recipeText[recipeText.index('Directions:\n') + 1:len(recipeText)]

def getMethodsDict(text, methodlist):
    pri_method = longestCookingMethod(text)
    all_methods = getVerbs(text)
    addi_method = isPrepMethod(methodlist, pri_method)
    methods_dict = {}
    methods_dict['primary'] = pri_method
    methods_dict['preparatory'] = addi_method
    return methods_dict

getMethodsDict(recipeDirecton, allmethods)

Recipe File already exists


{'primary': ['bake'],
 'preparatory': ['stir',
  'dilute',
  'reduce',
  'pare',
  'shuck',
  'braise',
  'stir-fry',
  'knead',
  'scald',
  'deseed',
  'knock back',
  'fold',
  'baste',
  'dust',
  'shallow-fry',
  'parboil',
  'steep',
  'score',
  'bruise',
  'season',
  'blanch',
  'whisk',
  'sauté',
  'caramelize',
  'cook',
  'simmer',
  'marinate',
  'dry-fry',
  'sear',
  'acidulate',
  'crumb',
  'macerate',
  'al dente',
  'poach',
  'broil',
  'devein',
  'blind',
  'beat',
  'grease',
  'julienne',
  'emulsify']}

In [57]:
example = '''Preheat oven to 400 degrees F (200 degrees C). 
Line a baking sheet with parchment paper.                            
Place cauliflower florets in a food processor; pulse until chopped into grains the size of rice.                            
Fill a large pot with about 1 inch of water; bring to a boil. Add cauliflower "rice"; cook until softened, 4 to 5 minutes. Drain in a fine sieve and cool until running water.  
                          
Pour cauliflower onto a clean dish towel; roll up and twist edges. Wring out as much water as possible.  
                          
Transfer cauliflower to a large bowl.
Add egg, mozzarella cheese, oregano, and salt; mix well. Spread on the lined baking sheet to about 1/2-inch thickness.                            
Bake in the preheated oven until firm to the touch and light golden brown in color, about 35 minutes.  '''

In [58]:
V1 = getVerbs(example.split("\n"))
V1

VBZ


['spread',
 'bake',
 'pulse',
 'line',
 'fill',
 'add',
 'cook',
 'roll',
 'pour',
 'drain',
 'cauliflower',
 'preheat',
 'wring',
 'mix',
 'bring',
 'transfer',
 'place']

In [59]:
isCookingMethod(V1, allmethods)

['bake', 'cook']

In [60]:
k = isCookingMethod(V1, allmethods)
isPrepMethod(V1 ,k)

['spread',
 'pulse',
 'fill',
 'line',
 'add',
 'roll',
 'pour',
 'drain',
 'cauliflower',
 'preheat',
 'wring',
 'mix',
 'bring',
 'transfer',
 'place']

In [61]:
allingredient = FormIngredientList('data/white-cheese-chicken-lasagna.txt')
mainingr = allingredient[0]['name']

{'name': 'lasagna noodles', 'quantity': '9', 'measurement': 'NoItem'}
{'name': 'butter', 'quantity': '1/2', 'measurement': 'cup'}
{'name': 'onion', 'quantity': '1', 'measurement': 'NoItem'}
{'name': 'garlic', 'quantity': '1', 'measurement': 'clove'}
{'name': 'flour', 'quantity': '1/2', 'measurement': 'cup'}
{'name': 'salt', 'quantity': '1', 'measurement': 'teaspoon'}
{'name': 'broth', 'quantity': '2', 'measurement': 'cup'}
{'name': 'milk', 'quantity': '1/2', 'measurement': 'cup'}
{'name': 'mozzarella cheese', 'quantity': '4', 'measurement': 'cup'}
{'name': 'Parmesan', 'quantity': '1', 'measurement': 'cup'}
{'name': 'basil', 'quantity': '1', 'measurement': 'teaspoon'}
{'name': 'oregano', 'quantity': '1', 'measurement': 'teaspoon'}
{'name': 'ground', 'quantity': '1/2', 'measurement': 'teaspoon'}
{'name': 'ricotta cheese', 'quantity': '2', 'measurement': 'cup'}
{'name': 'meat', 'quantity': '2', 'measurement': 'cup'}
{'name': 'spinach', 'quantity': '10', 'measurement': 'ounce'}
{'name': 'p

In [62]:
sentences = example.split("\n")
mainIngCookingMethod(mainingr, sentences)

[]

In [63]:
longestCookingMethod(example.split("\n"))

VBZ


['bake']

In [50]:
recipeDirecton

['Preheat oven to 425 degrees F (220 degrees C).\n',
 'Pull the edges of the split lobster shells apart and gently lift the tail meat to rest above the shells. Place the prepared lobster tails on a baking sheet.\n',
 'Brush each portion of tail meat with 1 teaspoon melted butter.\n',
 'Lightly mix the crushed crackers, crabmeat, 1/4 cup of clarified butter, parsley, seafood seasoning, garlic, lemon zest, lemon juice, salt, and white pepper in a bowl until thoroughly combined.\n',
 '"Spoon half the stuffing onto each lobster tail; press lightly to slightly shape the stuffing so it doesnt fall off.\n',
 'Bake the lobster tails in the preheated oven until the meat is opaque and the stuffing is golden brown on top, 10 to 12 minutes. An instant-read thermometer inserted into the thickest part of the lobster tail should read 145 degrees F (65 degrees C).\n']

In [51]:
allingredient = FormIngredientList('data/crab-stuffed-lobster-tail.txt')
#mainingr = allingredient[0]['name']
mainingr = 'lobster tails'
mainIngCookingMethod(mainingr, recipeDirecton)

{'name': 'lobster tails split', 'quantity': '2', 'measurement': 'NoItem'}
{'name': 'butter', 'quantity': '2', 'measurement': 'teaspoon'}
{'name': 'buttery round crackers', 'quantity': '15', 'measurement': 'NoItem'}
{'name': 'lump crabmeat', 'quantity': '1/2', 'measurement': 'cup'}
{'name': 'butter', 'quantity': '1/4', 'measurement': 'cup'}
{'name': 'parsley leaves', 'quantity': '1', 'measurement': 'tablespoon'}
{'name': 'seafood', 'quantity': '1', 'measurement': 'teaspoon'}
{'name': 'garlic', 'quantity': '1', 'measurement': 'clove'}
{'name': 'lemon zest', 'quantity': '1', 'measurement': 'teaspoon'}
{'name': 'lemon juice', 'quantity': '1', 'measurement': 'tablespoon'}
{'name': 'salt', 'quantity': '1/4', 'measurement': 'teaspoon'}
{'name': 'pepper', 'quantity': '1/4', 'measurement': 'teaspoon'}
[{'name': 'lobster tails split', 'quantity': '2', 'measurement': 'NoItem'}, {'name': 'butter', 'quantity': '2', 'measurement': 'teaspoon'}, {'name': 'buttery round crackers', 'quantity': '15', 'me

['bake']

In [74]:
getVerbs(recipeDirecton)

['preheat',
 'bake',
 'mix',
 'brush',
 'read',
 'press',
 'fall',
 'lift',
 'pull',
 'shape',
 '``',
 'doesnt',
 'place',
 'rest']

In [73]:
nltk.pos_tag('"Spoon half the stuffing onto each lobster tail; press lightly to slightly shape the stuffing so it doesnt fall off.\n'.split())

[('"Spoon', 'RB'),
 ('half', 'PDT'),
 ('the', 'DT'),
 ('stuffing', 'VBG'),
 ('onto', 'IN'),
 ('each', 'DT'),
 ('lobster', 'NN'),
 ('tail;', 'NN'),
 ('press', 'NN'),
 ('lightly', 'RB'),
 ('to', 'TO'),
 ('slightly', 'RB'),
 ('shape', 'VB'),
 ('the', 'DT'),
 ('stuffing', 'NN'),
 ('so', 'IN'),
 ('it', 'PRP'),
 ('doesnt', 'VBZ'),
 ('fall', 'NN'),
 ('off.', 'NN')]